In [6]:
import requests


API_KEY = 'e0a1bf2c844edb9084efc764c089dd748676cc14'
API_URL_STATIONS = 'https://api.jcdecaux.com/vls/v3/stations'
API_URL_CONTRACTS = 'https://api.jcdecaux.com/vls/v3/contracts'


params = {'apiKey': API_KEY}



response = requests.get(API_URL_STATIONS,params=params)
print(response.status_code)

if response.status_code == 200:
    stations = response.json()
    print('Nombre de station : ', len(stations))
else:
    print("Erreur en cherchant les données:", response.status_code)

200
Nombre de station :  2378


In [2]:
print(stations[0])
    

{'number': 55, 'contractName': 'toulouse', 'name': '00055 - SAINT-SERNIN - G. ARNOULT', 'address': '2 RUE GATIEN ARNOULT', 'position': {'latitude': 43.609022, 'longitude': 1.441105}, 'banking': True, 'bonus': False, 'status': 'OPEN', 'lastUpdate': '2024-04-19T23:25:58Z', 'connected': True, 'overflow': False, 'shape': None, 'totalStands': {'availabilities': {'bikes': 8, 'stands': 7, 'mechanicalBikes': 8, 'electricalBikes': 0, 'electricalInternalBatteryBikes': 0, 'electricalRemovableBatteryBikes': 0}, 'capacity': 15}, 'mainStands': {'availabilities': {'bikes': 8, 'stands': 7, 'mechanicalBikes': 8, 'electricalBikes': 0, 'electricalInternalBatteryBikes': 0, 'electricalRemovableBatteryBikes': 0}, 'capacity': 15}, 'overflowStands': None}


In [8]:
from collections import defaultdict

In [9]:

#Créons un dictionnaire pour stocker  les données facilement
city_stats = defaultdict(lambda: {'mechanical': 0, 'electrical':0,'total_velo':0})


#Parcours des donnees pour collecter les champs de statistiques utile
for station in stations: 
    city = station['contractName']
    velo_mechaniques = station['totalStands']['availabilities']['mechanicalBikes']
    velo_electrique = station['totalStands']['availabilities']['electricalBikes']
    
    #On ajoute nouveau valeurs aux valeur existant pour chaque ville
    city_stats[city]['mechanical'] += velo_mechaniques
    city_stats[city]['electrical'] += velo_electrique
    city_stats[city]['total_velo'] += (velo_mechaniques + velo_electrique)
    

In [10]:
print("Affichage des villes avec le plus de vélo en indiquant le pourcentage de chaque type de velo par ville")

#On trie les villes avec nombre de vélos totale dans cette ville
city_stats_sorted = sorted(city_stats.items(), key=lambda x: x[1]['total_velo'], reverse=True)



# Calcul du pourcentage de vélos mécaniques par rapport aux vélos électriques pour chaque ville
for city, stats in city_stats_sorted:
        total_velo = stats['total_velo']
        if total_velo > 0:
            pourcentage_mechanique = (stats['mechanical'] / total_velo) * 100
            pourcentage_electrique = 100 - pourcentage_mechanique
            print(f"Pourcentage à {city}: Velo_mechanique:{pourcentage_mechanique:.2f}% | Velo_electrique:{pourcentage_electrique:.2f}% | Total Velo: {total_velo}")
        else:
            print(f"Aucun vélo trouvé à {city}")

Affichage des villes avec le plus de vélo en indiquant le pourcentage de chaque type de velo par ville
Pourcentage à lyon: Velo_mechanique:56.15% | Velo_electrique:43.85% | Total Velo: 4055
Pourcentage à bruxelles: Velo_mechanique:58.40% | Velo_electrique:41.60% | Total Velo: 3779
Pourcentage à toulouse: Velo_mechanique:100.00% | Velo_electrique:0.00% | Total Velo: 2540
Pourcentage à seville: Velo_mechanique:100.00% | Velo_electrique:0.00% | Total Velo: 2473
Pourcentage à valence: Velo_mechanique:100.00% | Velo_electrique:0.00% | Total Velo: 2418
Pourcentage à dublin: Velo_mechanique:65.27% | Velo_electrique:34.73% | Total Velo: 1382
Pourcentage à luxembourg: Velo_mechanique:0.00% | Velo_electrique:100.00% | Total Velo: 1144
Pourcentage à nantes: Velo_mechanique:100.00% | Velo_electrique:0.00% | Total Velo: 1019
Pourcentage à ljubljana: Velo_mechanique:100.00% | Velo_electrique:0.00% | Total Velo: 700
Pourcentage à vilnius: Velo_mechanique:100.00% | Velo_electrique:0.00% | Total Velo: 

In [11]:
import random
import folium

#On choisi aléatoirement une station
random_station = random.choice(stations)
station_name = random_station['name']
station_address = random_station['address']
station_lat = random_station['position']['latitude']
station_lng = random_station['position']['longitude']

# Création de la carte centrée sur une position aleatoire choisi a partir des données stations
map = folium.Map(location=[station_lat, station_lng], zoom_start=12)

# Parcours des données des stations pour ajouter des marqueurs sur la carte
for station in stations:
    station_name = station['name']
    station_address = station['address']
    station_ville = station['contractName']
    station_lat = station['position']['latitude']
    station_lng = station['position']['longitude']

    # Création d'un marqueur pour chaque station avec des informations supplémentaires dans la popup
    folium.Marker(
        location=[station_lat, station_lng],
        popup=f"<b>{station_name}</b><br>{station_address},{station_ville}",
        tooltip=station_name
    ).add_to(map)

# # sauvegarde de la carte
map.save('stations_map.html')
#Affichage dans  un navigateur (ou en utilisant un serveur local)

